In [5]:
import pandas as pd
import numpy as np

In [3]:
df_respuestas = pd.read_csv("respuestas.csv")

In [4]:
df_respuestas.head()

,Marca temporal,Información y consentimiento,Edad,Experiencia previa con dashboards de datos,Rol principal,Respuesta,Dificultad,Respuesta.1,Dificultad.1,Respuesta.2,...,Me ha resultado fácil usar los paneles.,Creo que necesitaría ayuda técnica para usarlos.,Las funcionalidades están bien integradas.,Hay demasiada inconsistencia en los paneles.,Pienso que la mayoría de la gente aprendería a usarlos rápidamente,Los paneles me resultan muy engorrosos de usar.,Me siento seguro/a usando los paneles.,Necesité aprender muchas cosas antes de empezar a utilizarlos.,¿Qué es lo que más te ha gustado de los paneles?,¿Qué mejorarías o te ha resultado confuso?
0,28/06/2025 18:10:44,Acepto participar de forma voluntaria y que mi...,25 - 34 años,Sí,Profesional de los Datos,38,1,10,2,27,...,3,2,2,3,3,3,3,3,Que están bien historiados,Nada


In [7]:
# 2. Limpiar nombres de columnas
df = df_respuestas.copy()
df.columns = df.columns.str.strip()

In [8]:
df.head()

,Marca temporal,Información y consentimiento,Edad,Experiencia previa con dashboards de datos,Rol principal,Respuesta,Dificultad,Respuesta.1,Dificultad.1,Respuesta.2,...,Me ha resultado fácil usar los paneles.,Creo que necesitaría ayuda técnica para usarlos.,Las funcionalidades están bien integradas.,Hay demasiada inconsistencia en los paneles.,Pienso que la mayoría de la gente aprendería a usarlos rápidamente,Los paneles me resultan muy engorrosos de usar.,Me siento seguro/a usando los paneles.,Necesité aprender muchas cosas antes de empezar a utilizarlos.,¿Qué es lo que más te ha gustado de los paneles?,¿Qué mejorarías o te ha resultado confuso?
0,28/06/2025 18:10:44,Acepto participar de forma voluntaria y que mi...,25 - 34 años,Sí,Profesional de los Datos,38,1,10,2,27,...,3,2,2,3,3,3,3,3,Que están bien historiados,Nada


In [9]:
# 3. Distribuciones demográficas
print("=== Demográficos ===\n")
for col in ['Edad', 'Experiencia previa con dashboards de datos', 'Rol principal']:
    print(f"{col!r}:")
    print(df[col].value_counts(dropna=False), "\n")

# 4. Resumen de tareas (T1–T4)
tasks = {
    'T1': {'resp':'Respuesta',     'diff':'Dificultad',     'label':'Mediana de edad (2005)'},
    'T2': {'resp':'Respuesta.1',   'diff':'Dificultad.1',   'label':'% PIB pensiones (2022)'},
    'T3': {'resp':'Respuesta.2',   'diff':'Dificultad.2',   'label':'% <35 con vivienda (2024)'},
    'T4': {'resp':'Respuesta.3',   'diff':'Dificultad.3',   'label':'VPO terminadas (2001)'},
}

=== Demográficos ===

'Edad':
Edad
25 - 34 años    1
Name: count, dtype: int64 

'Experiencia previa con dashboards de datos':
Experiencia previa con dashboards de datos
Sí    1
Name: count, dtype: int64 

'Rol principal':
Rol principal
Profesional de los Datos    1
Name: count, dtype: int64 



In [12]:
# 3. Definir las columnas de respuesta y dificultad para cada tarea
tasks = {
    'T1': {'resp_col':'Respuesta',   'diff_col':'Dificultad',   'label':'Mediana de edad (2005)'},
    'T2': {'resp_col':'Respuesta.1', 'diff_col':'Dificultad.1', 'label':'% PIB pensiones (2022)'},
    'T3': {'resp_col':'Respuesta.2', 'diff_col':'Dificultad.2', 'label':'% <35 con vivienda (2024)'},
    'T4': {'resp_col':'Respuesta.3', 'diff_col':'Dificultad.3', 'label':'VPO terminadas (2001)'},
}

# 4. Extraer la "fila de prueba" (asumimos que es la primera, índice 0)
test_row = df.iloc[0]

# 5. Calcular para cada tarea si cada participante (a partir de fila 1) acertó o no
results = []
for t, cfg in tasks.items():
    # valor correcto de la prueba
    correct_val = pd.to_numeric(test_row[cfg['resp_col']], errors='coerce')
    
    # respuestas de los usuarios (omitimos la fila 0)
    resp = pd.to_numeric(df[cfg['resp_col']].iloc[1:], errors='coerce')
    diff = pd.to_numeric(df[cfg['diff_col']].iloc[1:], errors='coerce')
    
    # acierto exacto
    is_correct = resp == correct_val
    
    # estadísticas útiles
    n = len(resp)
    n_correct = is_correct.sum()
    pct_correct = 100 * n_correct / n if n else np.nan
    avg_diff_correct = diff[is_correct].mean() if n_correct else np.nan
    avg_diff_incorrect = diff[~is_correct].mean() if n_correct < n else np.nan
    
    results.append({
        'Tarea': t,
        'Descripción': cfg['label'],
        'N participantes': n,
        'Aciertos': f"{n_correct} ({pct_correct:.1f}%)",
        'Dif. media (acertaron)': round(avg_diff_correct, 2),
        'Dif. media (fallaron)': round(avg_diff_incorrect, 2)
    })

summary_df = pd.DataFrame(results)

# 6. Mostrar resultados
print("=== Resumen de aciertos y dificultad por tarea ===")
print(summary_df.to_string(index=False))

# 7. (Opcional) Correlación entre dificultad y acierto
print("\n=== Correlaciones dificultad vs. acierto ===")
for t, cfg in tasks.items():
    resp = pd.to_numeric(df[cfg['resp_col']].iloc[1:], errors='coerce')
    diff = pd.to_numeric(df[cfg['diff_col']].iloc[1:], errors='coerce')
    correct = (resp == pd.to_numeric(test_row[cfg['resp_col']], errors='coerce')).astype(int)
    corr = diff.corr(correct)
    print(f"{t}: Corr(dificultad, acierto) = {corr:.2f}")

=== Resumen de aciertos y dificultad por tarea ===
Tarea               Descripción  N participantes Aciertos  Dif. media (acertaron)  Dif. media (fallaron)
   T1    Mediana de edad (2005)                0 0 (nan%)                     NaN                    NaN
   T2    % PIB pensiones (2022)                0 0 (nan%)                     NaN                    NaN
   T3 % <35 con vivienda (2024)                0 0 (nan%)                     NaN                    NaN
   T4     VPO terminadas (2001)                0 0 (nan%)                     NaN                    NaN

=== Correlaciones dificultad vs. acierto ===
T1: Corr(dificultad, acierto) = nan
T2: Corr(dificultad, acierto) = nan
T3: Corr(dificultad, acierto) = nan
T4: Corr(dificultad, acierto) = nan


In [11]:
# 5. Cálculo de SUS
sus_cols = [
    'Me gustaría utilizar estos paneles con frecuencia.',
    'Los paneles son innecesariamente complejos.',
    'Me ha resultado fácil usar los paneles.',
    'Creo que necesitaría ayuda técnica para usarlos.',
    'Las funcionalidades están bien integradas.',
    'Hay demasiada inconsistencia en los paneles.',
    'Pienso que la mayoría de la gente aprendería a usarlos rápidamente',
    'Los paneles me resultan muy engorrosos de usar.',
    'Me siento seguro/a usando los paneles.',
    'Necesité aprender muchas cosas antes de empezar a utilizarlos.'
]
sus_cols = [c.strip() for c in sus_cols]
sus = df[sus_cols].astype(int).copy()

# Recodificar: ítems impares (positivos) → x−1; pares (negativos) → 5−x
for i, col in enumerate(sus_cols, start=1):
    if i % 2 == 1:
        sus[col] = sus[col] - 1
    else:
        sus[col] = 5 - sus[col]

df['SUS'] = sus.sum(axis=1) * 2.5

print("=== Estadísticos SUS ===")
print(df['SUS'].describe(), "\n")

# 6. Guardar resúmenes en CSV
task_df.to_csv('summary_tareas.csv', index=False)
df[['SUS']].to_csv('summary_sus.csv', index=False)

print("📊 Resúmenes guardados en summary_tareas.csv y summary_sus.csv")

=== Estadísticos SUS ===
count     1.0
mean     50.0
std       NaN
min      50.0
25%      50.0
50%      50.0
75%      50.0
max      50.0
Name: SUS, dtype: float64 

📊 Resúmenes guardados en summary_tareas.csv y summary_sus.csv


Los ítems del SUS se responden de 1 a 5, pero el resultado final **no** se queda en esa escala sino que se convierte a **0–100**. Así es como funciona:

1. Cada pregunta responde con un valor 1–5 (1 = Muy en desacuerdo; 5 = Muy de acuerdo).
2. Se recodifican:

   * Ítems positivos (1, 3, 5, 7, 9): se resta 1 → quedan en 0–4.
   * Ítems negativos (2, 4, 6, 8, 10): se resta su valor a 5 → quedan en 0–4.
3. Se suman todos esos valores (máximo 20) y se multiplican por **2.5** → escala 0–100.

Por tanto:

* Un **SUS = 50** equivale a una puntuación media en los ítems de:

  $$
    \frac{50}{2{.}5} \;=\; 20 \quad\Longrightarrow\quad \frac{20}{5\;\text{ítems}} = 4
  $$

  ¡Ojo! Un 20 sobre 20 significa un **2** de media antes de multiplicar (no un 4). El cálculo exacto es:

  * Si todas las preguntas fueran “3 = Neutral”:

    * Ítems positivos: 3 − 1 = 2 (por cada uno, 5 ítems → 5·2 = 10)
    * Ítems negativos: 5 − 3 = 2 (otros 5 ítems → 5·2 = 10)
    * Suma = 20 → 20·2.5 = 50

  Así que **SUS=50** corresponde a un **3 “Neutral”** de media en la escala original 1–5.

---

### Cómo interpretarlo

* **< 50** → peor que “Neutral” (la gente tiende al desacuerdo).
* **≈ 50** → “Neutral” (ni de acuerdo ni en desacuerdo).
* **> 50** → más hacia “De acuerdo”.
* **68** es el punto de corte estándar para decir que la usabilidad es “aceptable”.
* **80+** se considera “excelente”.

Con un **50**, tus testers han valorado los paneles de forma básicamente neutra, señalando que **hay margen de mejora** para llevar la experiencia hacia el “De acuerdo” (valores > 50).
